In [1]:
import tensorflow as tf
import os
from easydict import EasyDict
import numpy as np
from dataset import CaptchData
from model import Net, Model, MyEstimator
from generate_captch import CaptchaGenerator
import matplotlib.pyplot as plt

/home/renhui/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/renhui/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/renhui/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/renhui/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
config = EasyDict({
    #data
    'batch_size':64,
    'char_set':['digits','letters'],
    'text_lengths':[4,5,6,7],
    'image_shape':(64,256),
    #net
    'regularaztion_rate':0.0001,
    'num_hidden':128,
    'num_classes':10+26,
    'seq_len':16,#经过卷积层后，特征图的长度 计算得出
    #model
    'learning_rate':1e-3,
    'input_tensor_shape':(-1,64,256,3),
    'moving_average_decay':0.99,
    #estimator
    'model_path':'/home/renhui/File/model/captcha_ocr/model2/',
    'log_path':'/home/renhui/File/model/captcha_ocr/model2/log/',
    'max_steps':30000,
    'display_step':100,
    'save_step':2000,
    'eval_step':500,
})

In [3]:
data = CaptchData(config)
net = Net(config)
model = Model(config,net)
estimator = MyEstimator(model,config)

In [5]:
estimator.train(data.train_input_fn)

### 预测

In [6]:
def image_preprocess(image):
    image = image*(2./255)-1
    image = np.reshape(image, [1,64,256,3])
    return image

In [7]:
generator = CaptchaGenerator(char_set=['digits','letters'], shape=(64,256))

In [8]:
image1, label1, text1 = generator.create_sample()
image1 = image_preprocess(image1)
image2, label2, text2 = generator.create_sample()
image2 = image_preprocess(image2)
images = np.concatenate([image1, image2])

In [9]:
print(text1)
print(label1)
print(text2)
print(label2)

087R
[0, 8, 7, 27]
v556IFr
[31, 5, 5, 6, 18, 15, 27]


In [10]:
estimator.prepare_inference(batch_size=2)

INFO:tensorflow:Restoring parameters from /home/renhui/File/model/captcha_ocr/model2/model.ckpt-10000
Find model_file successfully！


True

In [11]:
pred, prob = estimator.inference(images)

In [12]:
pred

array([[ 0,  8,  7, 27,  0,  0,  0],
       [31,  5,  5,  6, 18, 15, 27]])

In [24]:
log_prob

array([[-0.00141811],
       [-0.00479698]], dtype=float32)

In [23]:
tf.sparse_to_dense(decoded[0][0], decoded[0][2], decoded[0][1])

<tf.Tensor 'SparseToDense:0' shape=(2, 7) dtype=int64>

In [30]:
pow(2,log_prob)

array([[0.99901754],
       [0.9966805 ]], dtype=float32)

In [13]:
?tf.sparse_tensor_to_dense